# Multi-label image classification with ImageDataGenerator

In [107]:
# sklearn
from sklearn.model_selection import train_test_split

# tensorflow and keras
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models

In [121]:
DIRECTORY = "/ /" # Path to the directory which contains CSVs and the folder 'images'
CSV_NAME = "dataset.csv/" # Name of the CSV we want to use
NUM_MOVEMENT = 27 # Number of movements to classify
NUM_GENRE = 10 # Number of genres to classify
IMG_HEIGHT = IMG_WIDTH = 224 # Model's inputs shapes


'''----------------------------------
Load the CSV
----------------------------------'''
df = pd.read_csv(DIRECTORY + "dataset.csv/")
assert type(df) == type(pd.DataFrame()) # Check if we created a dataframe
assert df.iloc[:, 1].nunique() ==  NUM_MOVEMENT # Check if we have the correct number of movements
assert df.iloc[:, 2].nunique() ==  NUM_GENRE # Check if we have the correct number of genres


'''----------------------------------
Train, test, val split
----------------------------------'''
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)
df_train, df_val = train_test_split(df_train, test_size=0.2, shuffle=True)
assert type(df_train) == type(pd.DataFrame()) # Check if we created dataframes
assert type(df_test) == type(pd.DataFrame())
assert type(df_val) == type(pd.DataFrame())


'''----------------------------------
Setup outputs columns
----------------------------------'''
assert len(list(test_df.columns[1:])) == 2
columns=list(test_df.columns[1:])


'''----------------------------------
Train ImageDataGenerator
----------------------------------'''
train_datagen = ImageDataGenerator( # This generator is only used for train data because it has data augmentation and we do not want to augment data from the test or val set
    rescale=1./255,
    rotation_range=15,
    zoom_range=(0.95, 0.95),
    horizontal_flip=True,
    dtype=tf.float32)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train, # Dataset used to get the path (column filename) and the linked outputs
    directory=DIRECTORY + "images/", # Path to the images
    x_col="filename", # Column with the name of the images that the generator will get from the directory
    y_col=columns, # Columns with the output of the images that the generator will get from the csv
    batch_size=32,
    seed=None,
    shuffle=True,
    class_mode="raw", # numpy array of values in y_col columns
    target_size=(IMG_HEIGTH, IMG_WIDTH)) # Resize the images to the input shape of the model

assert type(train_datagen) == type(ImageDataGenerator())


'''----------------------------------
Test and Val ImageDataGenerator
----------------------------------'''
test_val_datagen = ImageDataGenerator(  # We use a new generator without data augmentation
    rescale=1./255)

val_generator = test_val_datagen.flow_from_dataframe(
    dataframe=df_val, 
    directory=DIRECTORY + "images/",
    x_col="filename",
    y_col=columns,
    batch_size=32,
    seed=None,
    shuffle=False,
    class_mode="raw", 
    target_size=(IMG_HEIGTH, IMG_WIDTH))

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=DIRECTORY + "images/",
    x_col="filename",
    batch_size=1,
    seed=None,
    shuffle=False,
    class_mode=None, # No targets are returned (the generator will only yield batches of image data, which is useful to use in model.predict()
    target_size=(IMG_HEIGTH, IMG_WIDTH))


assert type(test_val_datagen) == type(ImageDataGenerator())



FileNotFoundError: ignored

In [ ]:
'''----------------------------------
Create model (Functional API)
----------------------------------'''
# Model tout pourri pour l'instant, transfert learning ensuite
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))
x = layers.Conv2D(16, 3, padding='same', activation='relu'),
x = layers.MaxPooling2D(),
x = layers.Conv2D(32, 3, padding='same', activation='relu'),
x = layers.MaxPooling2D(),
x = layers.Conv2D(64, 3, padding='same', activation='relu'),
x = layers.MaxPooling2D(),
x = layers.Dropout(rate=0.3),
x = layers.Conv2D(128, 3, padding='same', activation='relu'),
x = layers.MaxPooling2D(),
x = layers.Dropout(rate=0.4),
x = layers.Flatten(),
x = layers.Dense(128, activation='relu'),
x = layers.Dropout(rate=0.5),

output1 = Dense(NUM_MOVEMENT, activation = 'softmax')(x)
output2 = Dense(NUM_GENRE, activation = 'softmax')(x)

model = Model(inputs, [output1,output2]) # Create the model with 2 outputs


'''----------------------------------
Compile model
----------------------------------'''
model.compile(optimizer='adamax', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy', 'accuracy'])


'''----------------------------------
Fit model
----------------------------------'''
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size # see steps_per_epoch comment below
STEP_SIZE_VAL = val_generator.n//val_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN, # No 'batch_size' parameter for data coming from a generator or tf.Dataset, common choixe is to use num_samples // batch_size
    epochs=50,
    validation_data=val_generator,
    validation_steps=STEP_SIZE_VAL)


'''----------------------------------
Evaluate
----------------------------------'''
test_generator.reset() # Need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.
pred = model.evaluate(
    test_generator,
    steps=STEP_SIZE_TEST)


'''----------------------------------
Predict
----------------------------------'''
test_generator.reset() # Need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.
pred = model.predict(
    test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
